In [1]:
import pyspark
from delta import configure_spark_with_delta_pip

# Prepare the Spark builder
builder = pyspark.sql.SparkSession.builder.appName('Project 3') \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "100") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder,extra_packages=["graphframes:graphframes:0.8.4-spark3.5-s_2.12"]).getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions", spark._sc.defaultParallelism)

spark.conf.set("spark.sql.repl.eagerEval.enabled",True) # OK for exploration, not great for performance
spark.conf.set("spark.sql.repl.eagerEval.truncate", 500)

In [2]:
from graphframes import GraphFrame
from pyspark.sql.functions import coalesce, col, lit, sum as sql_sum, when, count
from pyspark.sql.functions import sum as spark_sum
from pyspark.sql import Window

In [3]:
df = spark.read.csv('./data/2009.csv', header=True)

In [4]:
# airports
vertices = df.selectExpr("ORIGIN as id").union(df.selectExpr("DEST as id")).distinct()
vertices.cache()

# flight is a directed edge from start airport (ORIGIN) to destination airport (DEST)
edges = df.selectExpr("ORIGIN as src", "DEST as dst")
edges.cache()

gf = GraphFrame(vertices, edges)

In [5]:
gf.persist()

GraphFrame(v:[id: string], e:[src: string, dst: string])

## Query 1

In [6]:
# count all flights from destination , rename columns
in_degree = gf.edges.groupBy("dst").count().withColumnRenamed("dst", "id").withColumnRenamed("count", "inDegree") # actual 

# add check to validate if they match 
in_degree_test = gf.inDegrees # for testing expected results

comparison = in_degree_test.alias("expected") \
    .join(in_degree.alias("actual"), on="id", how="outer")

in_degrees_match = comparison.filter(
    "expected.inDegree != actual.inDegree OR expected.inDegree IS NULL OR actual.inDegree IS NULL"
).count() == 0
print(in_degrees_match)

#in_degree.orderBy("inDegree", ascending=True).show()
in_degree.show()

True
+---+--------+
| id|inDegree|
+---+--------+
|JAX|   28813|
|BOS|  110463|
|MEM|   71721|
|XNA|   13764|
|BTV|    6021|
|SYR|    9330|
|JFK|  119571|
|MBS|    3443|
|SBN|    4527|
|HPN|   10661|
|EKO|    1416|
|ANC|   17788|
|VLD|    1046|
|DAB|    2149|
|EVV|    4761|
|BGM|     677|
|KTN|    2373|
|WRG|     722|
|MAF|    6526|
|MIA|   63949|
+---+--------+
only showing top 20 rows



In [7]:
# count of flights from each origin, rename columns
out_degree = gf.edges.groupBy("src").count().withColumnRenamed("src", "id").withColumnRenamed("count", "outDegree")

# check for validaiton if they match
out_degree_test = gf.outDegrees # expected values

comparison_out = out_degree_test.alias("expected") \
    .join(out_degree.alias("actual"), on="id", how="outer")

out_degrees_match = comparison_out.filter(
    "expected.outDegree != actual.outDegree OR expected.outDegree IS NULL OR actual.outDegree IS NULL"
).count() == 0
print(out_degrees_match)

#out_degree.orderBy("outDegree", ascending=True).show()
out_degree.show()

True
+---+---------+
| id|outDegree|
+---+---------+
|JAX|    28810|
|MEM|    71713|
|BTV|     6028|
|BOS|   110460|
|XNA|    13755|
|SYR|     9336|
|JFK|   119574|
|MBS|     3444|
|SBN|     4526|
|HPN|    10657|
|EKO|     1416|
|ANC|    17791|
|VLD|     1046|
|DAB|     2146|
|EVV|     4761|
|BGM|      678|
|KTN|     2374|
|WRG|      722|
|MAF|     6523|
|MIA|    63943|
+---+---------+
only showing top 20 rows



In [8]:
# compute the total degree of each airport ( sum of in-degree and out-degree)
degree = in_degree.join(out_degree, "id", "outer") \
    .withColumn("inDegree", coalesce(in_degree["inDegree"], pyspark.sql.functions.lit(0))) \
    .withColumn("outDegree", coalesce(out_degree["outDegree"], pyspark.sql.functions.lit(0))) \
    .withColumn("totalDegree", pyspark.sql.functions.col("inDegree") + pyspark.sql.functions.col("outDegree"))

#degree.orderBy("totalDegree", ascending=True).show()
degree.show()

+---+--------+---------+-----------+
| id|inDegree|outDegree|totalDegree|
+---+--------+---------+-----------+
|ABE|    4037|     4034|       8071|
|ABI|    2490|     2490|       4980|
|ABQ|   35577|    35582|      71159|
|ABY|     997|      995|       1992|
|ACK|     343|      342|        685|
|ACT|    1052|     1053|       2105|
|ACV|    3364|     3370|       6734|
|ACY|     522|      522|       1044|
|ADK|     103|      103|        206|
|ADQ|     631|      631|       1262|
|AEX|    2948|     2947|       5895|
|AGS|    3106|     3107|       6213|
|AKN|      77|       77|        154|
|ALB|   12020|    12018|      24038|
|ALO|     331|      330|        661|
|AMA|    6649|     6649|      13298|
|ANC|   17788|    17791|      35579|
|ASE|    4708|     4701|       9409|
|ATL|  417457|   417449|     834906|
|ATW|    5306|     5303|      10609|
+---+--------+---------+-----------+
only showing top 20 rows



In [9]:
from pyspark.sql import functions as F
# make sure direction doesn't matter
edges_undirected = edges.select(
    F.when(F.col("src") < F.col("dst"), F.col("src")).otherwise(F.col("dst")).alias("src"),
    F.when(F.col("src") < F.col("dst"), F.col("dst")).otherwise(F.col("src")).alias("dst")
).distinct()

# find A -> B -> C, create three copies of edge list to use them for creating triangles
e1 = edges_undirected.alias("e1")
e2 = edges_undirected.alias("e2")
e3 = edges_undirected.alias("e3")

# form the triangles
triangles = (
    e1.join(e2, F.col("e1.dst") == F.col("e2.src"))  # join to find flights from a - > b an b -> c
      .join(e3, (F.col("e3.src") == F.col("e1.src")) & (F.col("e3.dst") == F.col("e2.dst")))  # join to check if there is edge between a -> c
      .select(F.col("e1.src").alias("a"), F.col("e1.dst").alias("b"), F.col("e2.dst").alias("c")) # select nodes
      .filter("a < b AND b < c") # keep each triangle once
)

# count the triangles for each airport
vertices_in_triangles = triangles.selectExpr("a as id").union(triangles.selectExpr("b as id")).union(triangles.selectExpr("c as id"))
triangles_actual = vertices_in_triangles.groupBy("id") \
    .agg(F.count("*").alias("count"))  

#check if matches with graphframes built in function
triangles_test = gf.triangleCount()

comparison = triangles_test.select("id", F.col("count").alias("test_count")) \
    .join(triangles_actual.select("id", F.col("count").alias("actual_count")), on="id", how="outer")
are_equal = comparison.filter(F.col("test_count") != F.col("actual_count")).isEmpty()
print(are_equal)

#triangles_actual.orderBy("count", ascending=False).show(truncate=False)
triangles_actual.select("id", "count").show()

True
+---+-----+
| id|count|
+---+-----+
|BOS|  860|
|ANC|  122|
|BTV|   34|
|JFK|  942|
|JAX|  342|
|MEM| 1105|
|HPN|   36|
|MBS|    6|
|EVV|   15|
|KTN|    4|
|CWA|    3|
|HLN|    8|
|BOI|   88|
|MIA|  728|
|PHX| 1191|
|SBA|   27|
|MSO|   11|
|FCA|   10|
|MFR|   17|
|PSP|   55|
+---+-----+
only showing top 20 rows



## Query 2

In [10]:
total_triangles = triangles_actual.select(spark_sum("count")).first()[0] // 3 # divide by 3 as each triangle has 3 airports

total_triangles_test = triangles_test.select(spark_sum("count")).first()[0] // 3 # divide by 3 as each triangle has 3 airports

print(f"Total number of triangles calculated manually: {total_triangles}")
print(f"Expected total number of triangles: {total_triangles_test}")

Total number of triangles calculated manually: 16015
Expected total number of triangles: 16015


## Query 3

In [11]:
# EigenCentrality
## https://cambridge-intelligence.com/keylines-faqs-social-network-analysis/

# Make undirected edges by adding reverse direction
reversed_edges = edges.selectExpr("dst as src", "src as dst")
undirected_edges = edges.union(reversed_edges).distinct()

scores = vertices.withColumn("score", lit(1.0))

In [12]:
num_iterations = 10

for i in range(num_iterations):
    # Join edges with current scores
    joined = undirected_edges.join(scores, undirected_edges.dst == scores.id, how="left") \
                              .select(undirected_edges.src.alias("id"), col("score"))

    # Sum neighbor scores to update each node
    updated_scores = joined.groupBy("id").agg(spark_sum("score").alias("score"))

    # Normalize scores to prevent divergence (optional)
    total = updated_scores.agg(spark_sum("score")).first()[0]
    normalized_scores = updated_scores.withColumn("score", col("score") / total)

    # Update for next iteration
    scores = normalized_scores

In [13]:
scores.orderBy(col("score").desc()).show()

+---+--------------------+
| id|               score|
+---+--------------------+
|ATL|0.018765722188918116|
|ORD|0.018108878438614258|
|DFW|0.016941191132151725|
|DTW| 0.01669891075140396|
|DEN|0.016140351072366586|
|MSP|0.016124197587719383|
|IAH|0.015634522037957406|
|CVG|0.015054412087523953|
|EWR|0.014559829904750299|
|LAS|0.014527065196091225|
|PHX|0.014275998919980493|
|MEM|0.014028033193475667|
|MCO|0.013546284087077538|
|CLT|0.013532499671015705|
|LAX| 0.01323284686945521|
|SLC|0.013181054625054962|
|BWI|0.013131956958904989|
|IAD|0.013019495597477422|
|CLE|  0.0129345700134493|
|PHL|0.012915917719506932|
+---+--------------------+
only showing top 20 rows



## Query 4

In [14]:
# Count the nodes
num_vertices = vertices.count()

# Pagerank to 1/N
ranks = vertices.withColumn("pagerank", lit(1.0 / num_vertices))

# Outdegree per node
out_degrees = edges.groupBy("src").agg(count("dst").alias("out_deg"))

# Join edges with outdegree data
edges_with_deg = edges.join(out_degrees, on="src", how="left")

# Parameters
reset_prob = 0.15
iterations = 5

# Compute pagerank
for i in range(iterations):
    # Join current ranks with edge list
    contribs = edges_with_deg.join(ranks, edges_with_deg.src == ranks.id) \
        .withColumn("contrib", col("pagerank") / col("out_deg")) \
        .select("dst", "contrib")

    # Sum contributions per destination node
    agg_contribs = contribs.groupBy("dst").agg(sql_sum("contrib").alias("total_contrib"))

    # Calculate new ranks using the damping formula
    ranks = agg_contribs.select(
        col("dst").alias("id"),
        (lit(reset_prob) / num_vertices + lit(1 - reset_prob) * col("total_contrib")).alias("pagerank")
    ).union(
        vertices.join(agg_contribs, vertices.id == agg_contribs.dst, "left_anti")
        .select("id").withColumn("pagerank", lit(reset_prob / num_vertices))
    )
ranks.orderBy("pagerank", ascending=False).show()

+---+--------------------+
| id|            pagerank|
+---+--------------------+
|ATL| 0.06589617957219522|
|ORD|0.044602712562445204|
|DFW| 0.04056298155959312|
|DEN| 0.03440153240671735|
|LAX|0.025932323684414206|
|IAH| 0.02463411828994618|
|SLC| 0.02436011954110491|
|DTW| 0.02410120320306513|
|PHX|0.023946028768846384|
|SFO|0.019935590729512627|
|MSP|0.019588031221911488|
|LAS| 0.01910365727310598|
|SEA|0.017718406483243555|
|JFK|0.015464365582573107|
|MCO|0.014825790376571609|
|EWR|0.014633426817575611|
|CLT| 0.01419770185600247|
|BOS|0.012804259442974615|
|BWI|0.012089968031306676|
|LGA| 0.01199432812756782|
+---+--------------------+
only showing top 20 rows



## Query 5

In [15]:
# Degree centrality (number of direct connections)
## Shows how connected an airport is
## Can be used to detect big airports or bottlenecks

degree_centrality = in_degree.join(out_degree, "id", "outer") \
    .withColumn("inDegree", coalesce(col("inDegree"), lit(0))) \
    .withColumn("outDegree", coalesce(col("outDegree"), lit(0))) \
    .withColumn("degreeCentrality", col("inDegree") + col("outDegree"))


degree_centrality.orderBy(col("degreeCentrality").desc()).show()

+---+--------+---------+----------------+
| id|inDegree|outDegree|degreeCentrality|
+---+--------+---------+----------------+
|ATL|  417457|   417449|          834906|
|ORD|  313769|   313848|          627617|
|DFW|  264398|   264396|          528794|
|DEN|  235700|   235675|          471375|
|LAX|  192916|   192879|          385795|
|PHX|  183491|   183502|          366993|
|IAH|  182088|   182097|          364185|
|LAS|  153984|   153993|          307977|
|DTW|  152075|   152081|          304156|
|SFO|  136532|   136488|          273020|
|SLC|  131674|   131694|          263368|
|MCO|  120936|   120944|          241880|
|MSP|  119759|   119732|          239491|
|JFK|  119571|   119574|          239145|
|EWR|  118602|   118602|          237204|
|CLT|  116640|   116650|          233290|
|BOS|  110463|   110460|          220923|
|SEA|  100922|   100948|          201870|
|BWI|  100928|   100923|          201851|
|LGA|  100323|   100334|          200657|
+---+--------+---------+----------